In [4]:
using CSV, DataFrames
using JuMP, HiGHS

three_stores = CSV.read("three_stores.csv", DataFrame)
price = three_stores.Price
sales = three_stores.Sales
advertising = three_stores.Advertising
model = Model(HiGHS.Optimizer)

@variable(model, x[1:3] >= 0)

@objective(model, Max, sum(price[i] * (sales[i] + 0.05 * x[i]) for i in 1:3))

@constraint(model, sum(x) <= 6000) #does not increase by more than 6000 for the 3
@constraint(model, x[1] >= 1000) #increase by 1000 at lease
@constraint(model, x[2] >= 1000)
@constraint(model, x[3] >= 1000)
@constraint(model, x[1] + 0.25 * x[2] <= 3000)  # Exposure limit for 1
@constraint(model, x[2] + 0.25 * (x[1] + x[3]) <= 3000)  # Exposure limit for 2
@constraint(model, x[3] + 0.25 * x[2] <= 3000)  # Exposure limit for 3

# Solve the model
optimize!(model)
println("Optimal advertising increase for each company: ", value.(x))
println("Optimal total revenue increase: ", objective_value(model))


Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-01, 1e+00]
  Cost   [4e+00, 6e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+03, 6e+03]
Presolving model
4 rows, 3 cols, 10 nonzeros  0s
4 rows, 3 cols, 10 nonzeros  0s
Presolve : Reductions: rows 4(-3); columns 3(-0); elements 10(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.5749988028e+01 Ph1: 4(7); Du: 3(15.75) 0s
          3    -3.4588440000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 3
Objective value     :  3.4588440000e+04
HiGHS run time      :          0.00
Optimal advertising increase for each company: [1500.0, 2000.0, 2500.0]
Optimal total revenue increase: 34588.44
